# February 06, 2024 - Session Notes

Multi Route Documentation

#### __(Step 1) - Router Template:__

`router_temp` is created by formatting the predefined `MULTI_PROMPT_ROUTER_TEMPLATE` with the `destination_str`.
The `MULTI_PROMPT_ROUTER_TEMPLATE` is a template designed for creating router chains that handle multiple prompts or input scenarios.
It dynamically specifies the available destinations (subchains) based on the provided `destination_str`.

#### __(Step 2) - Router Prompt Template:__

`router_prompt` is a dynamic prompt template for the router chain.
It uses the formatted `router_temp` as the template.
The template expects an input variable named "input" (which will be replaced with actual user input during formatting).
The `RouterOutputParser()` handles parsing the LLM’s response based on this prompt.

#### __(Step 3) - LLMRouterChain:__
`router_chain` is an instance of `LLMRouterChain`.
It uses the specified LLM (llm) and the `router_prompt`.
The router chain will evaluate user inputs using the LLM and route them to the appropriate subchain based on the LLM’s output.

Use Cases:
The resulting `router_chain` can be used in various scenarios:
- Chatbots: To handle diverse user queries effectively.
- Task Routing: For complex tasks where different subchains specialize in specific types of input.
- Dynamic Selection: Based on context or user input, the router chain dynamically selects the most relevant processing path.

In summary, this code snippet sets up a router chain that intelligently routes user inputs to different subchains based on the LLM’s output, enhancing the overall language model interaction experience. 🌐🔗

----------------------------------------------------------------------------------------------------------------------------------------------------------

LLMRouterChain requires base llm_chain prompt to have an output parser that converts LLM text output to a dictionary with keys 'destination' and 'next_inputs'. Received a prompt with no output parser. (type=value_error)

In [2]:
from langchain.chains import LLMChain
from langchain.chains.router import RouterChain, MultiRouteChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [3]:
model = ChatGoogleGenerativeAI(model='gemini-pro')

In [4]:
scientist = '''
You are a scientist.
You do not answer any question that is outside the fold of science concepts.
Here is the question for you. \n {input}
'''

mathematician = '''
You are a mathematician.
You do not answer any question that is outside the fold of mathematic concepts.
Here is the question for you. \n {input}
'''

lawyer = '''
You are a Lawyer. 
You do not answer any question that is outside the fold of law concepts.
Here is the question for you. \n {input}
'''

In [5]:
prompt_infos = [
    {
        "name" : "scientist",
        "desc" : "Rsponds to science questions only",
        "prompt_template" : scientist
    },
    {
        "name" : "mathematician",
        "desc" : "Responds to mathematics questions only",
        "prompt_template" : mathematician
    },
    {
        "name" : "Lawyer",
        "desc" : "Responds to law questions only",
        "prompt_template" : lawyer
    }
]

In [6]:
default_prompt = ChatPromptTemplate.from_template(template='{input}')
default_chain = LLMChain(llm=model, prompt=default_prompt)

In [7]:
destination_chains = {}

for _ in prompt_infos:
    name = _['name']
    templ = _['prompt_template']
    chat_prompt = ChatPromptTemplate.from_template(templ)
    chain = LLMChain(llm=model, prompt=chat_prompt)
    destination_chains[name] = chain

In [8]:
destination_chains

{'scientist': LLMChain(prompt=ChatPromptTemplate(input_variables=['input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='\nYou are a scientist.\nYou do not answer any question that is outside the fold of science concepts.\nHere is the question for you. \n {input}\n'))]), llm=ChatGoogleGenerativeAI(model='gemini-pro', client= genai.GenerativeModel(
    model_name='models/gemini-pro',
    generation_config={}.
    safety_settings={}
 ))),
 'mathematician': LLMChain(prompt=ChatPromptTemplate(input_variables=['input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='\nYou are a mathematician.\nYou do not answer any question that is outside the fold of mathematic concepts.\nHere is the question for you. \n {input}\n'))]), llm=ChatGoogleGenerativeAI(model='gemini-pro', client= genai.GenerativeModel(
    model_name='models/gemini-pro',
    generation_config={}.
    safety_settings={}
 ))),
 'Law

In [9]:
#routing destinations
destinations = [f"{_['name']} : {_['desc']}" for _ in prompt_infos]

In [10]:
destinations

['scientist : Rsponds to science questions only',
 'mathematician : Responds to mathematics questions only',
 'Lawyer : Responds to law questions only']

In [11]:
destinations_str = '\n'.join(destinations)

In [12]:
router_temp = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)

In [13]:
print(router_temp)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
scientist : Rsponds to science questions only
mathematician : Responds to mathematics questions only


In [14]:
router_prompt = PromptTemplate(
    template=router_temp, 
    input_variables=['input'], 
    output_parser=RouterOutputParser()
    )

router_chain = LLMRouterChain.from_llm(llm=model, prompt=router_prompt)

In [15]:
chain = MultiRouteChain(router_chain=router_chain, destination_chains=destination_chains, default_chain=default_chain, verbose=True )

In [16]:
result = chain.invoke("What is an atom?")
print(result['text'])



> Entering new MultiRouteChain chain...


C:\Users\mfurq\AppData\Roaming\Python\Python311\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


scientist: {'input': 'What is an atom?'}
> Finished chain.
An atom is the basic unit of matter and the defining structure of elements. It consists of a central nucleus surrounded by electrons. The nucleus is made of positively charged protons and neutral neutrons. The electrons are negatively charged and orbit the nucleus in shells. Different elements have different numbers of protons, which determines their atomic number. The atomic number also determines the number of electrons in an atom, since atoms are electrically neutral. Atoms can combine with each other to form molecules, which are the basic units of compounds.


In [17]:
result = chain.invoke("What is IPC 421?")
print(result['text'])



> Entering new MultiRouteChain chain...


C:\Users\mfurq\AppData\Roaming\Python\Python311\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Lawyer: {'input': 'What is IPC 421?'}
> Finished chain.
**IPC Section 421:** Dishonestly or fraudulently inducing delivery of property

**Ingredients of the offence**:
- Dishonest or fraudulent inducement: The accused must have induced the delivery of property by dishonestly or fraudulently. Dishonesty means the intention to deceive or cause wrongful gain or loss. Fraudulently means using deception or trickery to induce the delivery of property.
- Delivery of property: The accused must have induced the delivery of property. Property includes movable and immovable property, tangible and intangible property, and actionable claims.
- Knowledge or intention: The accused must have known or intended that the delivery of property would be induced by his/her dishonest or fraudulent inducement.

**Punishment**:
- Imprisonment for a term of up to three years, or
- Fine, or
- Both


In [18]:
result = chain.invoke("What is derivative of x with respect to y?")
print(result['text'])



> Entering new MultiRouteChain chain...


C:\Users\mfurq\AppData\Roaming\Python\Python311\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


mathematician: {'input': 'What is the derivative of x with respect to y?'}
> Finished chain.
The derivative of x with respect to y is 0, since x is a constant with respect to y.


<<< End Of Document >>>